# Infer Pooled PK Model Parameters from Individuals in Lung Cancer Treatment Group with Single Erlotinib Dose

## Preclinical data

In [1]:
import erlotinib as erlo
import pandas as pd

# Get data
data = erlo.DataLibrary().lung_cancer_single_erlotinib_dose_group()

# Create scatter plot
fig = erlo.plots.PKTimeSeriesPlot()
fig.add_data(data, biomarker='Plasma concentration')
fig.set_axis_labels(
    time_label=r'$\text{Time in day}$', 
    biom_label=r'$\text{Plasma concentration in ng/mL}$', 
    dose_label=r'$\text{Dose in mg}$')
fig.show()

**Figure 1:** Visualisation of the measured erlotinib plasma concentration in mice with patient-derived lung cancer implants.

## Build Population PK model

In [2]:
import erlotinib as erlo

# Define mechanistic model
path = erlo.ModelLibrary().one_compartment_pk_model()
mechanistic_model = erlo.PharmacokineticModel(path)
mechanistic_model.set_administration(compartment='central', direct=False)
mechanistic_model.set_parameter_names(names={
    'central.drug_amount': 'Initial plasma drug amount in mg',
    'dose.drug_amount': 'Initial dose comp. drug amount in mg',
    'central.size': 'Volume of distribution in L',
    'dose.absorption_rate': 'Absorption rate in 1/d',
    'myokit.elimination_rate': 'Elimination rate in 1/d'})

# Define error model
error_model = erlo.ConstantAndMultiplicativeGaussianErrorModel()

# Define population model
population_models = [
    erlo.PooledModel(),   # Volume of distribution
    erlo.PooledModel(),   # Absorption rate
    erlo.PooledModel(),   # Elimination rate
    erlo.PooledModel(),   # Plasma concentration sigma base
    erlo.PooledModel(),]  # Plasma concentration sigma rel.

# Build model
problem = erlo.ProblemModellingController(
    mechanistic_model, error_model)
problem.fix_parameters(name_value_dict={
    'Initial plasma drug amount in mg': 0,
    'Initial dose comp. drug amount in mg': 0})
problem.set_population_model(population_models)

## Prior predictive checks

In [3]:
import numpy as np
import pints

# Define prior distribution
log_priors = [
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Volume of distribution
    pints.HalfCauchyLogPrior(location=1, scale=2),     # Absorption rate
    pints.HalfCauchyLogPrior(location=0, scale=0.1),   # Elimination rate
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Sigma base
    pints.HalfCauchyLogPrior(location=0, scale=0.1)]   # Sigma rel.
log_prior = pints.ComposedLogPrior(*log_priors)

# Define prior predictive model and sample 1000 virtual patients
predictive_model = problem.get_predictive_model(exclude_pop_model=True)
model = erlo.PriorPredictiveModel(predictive_model, log_prior)
seed = 42
n_samples = 1000
times = np.linspace(start=0, stop=5, num=100)
model.set_dosing_regimen(2.5, start=0, period=4, num=2)
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma conc. in ng/mL}$')
fig.show()

**Figure 2:** Prior predictive model of erlotinib plasma concentration measurements over time. The shaded areas estimate the 0.3, 0.6 and 0.9 bulk probability of the prior predictive distribution by sampling 1000 virtual "measurements" at each of the time points.

## Find MAP estimates for model parameters

In [5]:
# Create posterior
problem.set_data(data, output_biomarker_dict={'central.drug_concentration': 'Plasma concentration'})
problem.set_log_prior(log_priors)
log_posterior = problem.get_log_posterior()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posterior)
opt.set_transform(transform=pints.LogTransformation(n_parameters=problem.get_n_parameters()))
opt.set_n_runs(1)
map_estimates = opt.run(show_run_progress_bar=True)

### Visualise optimisation results

In [6]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 3:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value, and the x axis the corresponding individual.

## Find posterior probability distribution

In [7]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posterior)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=problem.get_n_parameters()))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

### Visualise marginal posteriors

In [8]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 4:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples for each individual.

## Posterior predictive checks

In [10]:
# Define posterior predictive model
predictive_model = problem.get_predictive_model(exclude_pop_model=True)
model = erlo.PosteriorPredictiveModel(predictive_model, posterior_samples)
seed = 42
n_samples = 1000
times = np.linspace(start=0, stop=5, num=1000)
model.set_dosing_regimen(2.5, start=0, period=4, num=2)
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.add_data(data, biomarker='Plasma concentration')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Plasma conc. in ng/mL}$')
fig.show()

**Figure 5** Approximate posterior predictive model for the erlotinib plasma concentration in the population over time for different dosing strategies. The shaded area indicate the 90% bulk of the posterior predictive model. The posterior predictive model was approximated by sampling 1000 parameters from the posterior distribution, and subsequent sampling of 50 equidistant time points from the predictive model. The scatter points indicate the measurement data that was used for the inference.